In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


import tensorflow as tf

import re


In [6]:
import pdfplumber

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

resume_text = extract_text_from_pdf("/kaggle/input/resume-data/Shawn_resume.pdf")

In [7]:
resume_text

'Shawn Callistus\n7397366498 | shawncallistusj@gmail.com | linkedin.com/shawn | github.com/shawn\nEducation\nShiv Nadar University 2025\nBachelor of Technology in Artificial Intelligence and Data Science CGPA: 9.0\nSt. Michaels Academy 2021\n12th Grade, TNBHSE Percentage: 96.10\nExperience\nDataZymes Pvt Ltd Jan 2025 – June 2025\nData Engineer Associate Bangalore\n• Trained on AWS services\n• Built a data warehousing framework for landing, integration, dimension stages using PySpark and Snowflake\n• Redesigned and deployed an ETL news processing pipeline on Google Cloud with web scraping, LLM-based\nclassification, and clustering\n• Implemented an LLM-powered web scraper reducing manual workload by 90% along with a Knowledge Graph\nRAG pipeline using LangChain and Neo4j.\nHCL Technologies Ltd June 2024 – Aug 2024\nAI Intern Chennai\n• Developed a prototype for product defect detection system using OpenCV for automated fault identification.\n• Worked with Depth-based Stable Diffusion mo

In [9]:
resume_df = pd.read_csv("/kaggle/input/resume-data/Extracted_Data.csv")
resume_df.rename(columns={"Unnamed: 0" : "Id"}, inplace=True)
resume_df.head()

,Id,Category,Pages,Data
0,10554236,ACCOUNTANT,5,ACCOUNTANT\nSummary\nFinancial Accountant spec...
1,10674770,ACCOUNTANT,2,STAFF ACCOUNTANT\nSummary\nHighly analytical a...
2,11163645,ACCOUNTANT,2,ACCOUNTANT\nProfessional Summary\nTo obtain a ...
3,11759079,ACCOUNTANT,2,SENIOR ACCOUNTANT\nExperience\nCompany Name Ju...
4,12065211,ACCOUNTANT,2,SENIOR ACCOUNTANT\nProfessional Summary\nSenio...


In [10]:
resume_df = resume_df[resume_df["Data"].duplicated() == False]
resume_df.dropna(inplace=True)

In [11]:
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
stop_words = set(stopwords.words('english'))

def pre_process_text(string):
    
 
    string = re.sub(r'https?://\S+|www\.\S+', "", string)
    string = re.sub(r'[!\"#$%&\'()*+,-./:;<=>?@[\\\]]', "", string) 
    
    string = string.lower()
    
    processed_string = ""
    
    # Removing Stopwords
    for sent in sent_tokenize(string):
        sub_string = " ".join([word for word in word_tokenize(sent) if word not in stop_words])
        processed_string += sub_string + " "
        
    return processed_string

In [ ]:
stop_words = set(stopwords.words('english'))
def resume_data_feature_extraction(string):
    
    # Remove any Links [HyperLink / WebLink]
    string = re.sub(r'https?://\S+|www\.\S+', "", string) 
    
    # Convert string to Lowercase
    string = string.lower()
    
    mp = {
          "responsibilities" : [],
          "skill" : [],
          "education" : [],
          "experience" : [],
          "highlights" :[],
          "summary" :[],
         }
    
    for sent in sent_tokenize(string):
        for req in mp: 
                sub_string = " ".join([word for word in word_tokenize(sent) if word not in stop_words]) 
                mp[req].append(sub_string)
                break 
    
    return_string = ""
    for req in mp:
        if(len(mp[req]) == 0):
            continue
        return_string += " " + req + " " + " ".join(mp[req])
    
    return_string = re.sub(r'[!\"#$%&\'()*+,-./:;<=>?@[\\\]]', "", return_string) 
    
    return return_string

In [13]:
resume_df

,Id,Category,Pages,Data
0,10554236,ACCOUNTANT,5,ACCOUNTANT\nSummary\nFinancial Accountant spec...
1,10674770,ACCOUNTANT,2,STAFF ACCOUNTANT\nSummary\nHighly analytical a...
2,11163645,ACCOUNTANT,2,ACCOUNTANT\nProfessional Summary\nTo obtain a ...
3,11759079,ACCOUNTANT,2,SENIOR ACCOUNTANT\nExperience\nCompany Name Ju...
4,12065211,ACCOUNTANT,2,SENIOR ACCOUNTANT\nProfessional Summary\nSenio...
...,...,...,...,...
2479,86322251,TEACHER,2,READING TEACHER\nSummary\nI am a highly motiva...
2480,86597425,TEACHER,3,HISTORY TEACHER\nProfessional Summary\nTo be e...
2481,90363254,TEACHER,2,"TEACHER\nSummary\nHighly ethical, dependable, ..."
2482,96547039,TEACHER,2,TEACHER\nSummary\nTalented early education pro...


In [14]:
resume_df["Data"] = resume_df["Data"].astype(str)
resume_df["Processed_Data"] = resume_df["Data"].apply(pre_process_text)




In [16]:

resume_df["Processed_Feature"] = resume_df["Data"].apply(resume_data_feature_extraction)

resume_df.to_csv("Processed_Data.csv")

In [17]:

resumeDataSet = pd.read_csv('/kaggle/working/Processed_Data.csv' ,encoding='utf-8')
resumeDataSet['Processed_Data']=resumeDataSet['Processed_Data'].dropna()

resumeDataSet.head()

,Unnamed: 0,Id,Category,Pages,Data,Processed_Data,Processed_Feature
0,0,10554236,ACCOUNTANT,5,ACCOUNTANT\nSummary\nFinancial Accountant spec...,accountant summary financial accountant specia...,responsibilities briefing identified team mem...
1,1,10674770,ACCOUNTANT,2,STAFF ACCOUNTANT\nSummary\nHighly analytical a...,staff accountant summary highly analytical det...,skill excel problem solving strategic plannin...
2,2,11163645,ACCOUNTANT,2,ACCOUNTANT\nProfessional Summary\nTo obtain a ...,accountant professional summary obtain positio...,skill accountant professional summary obtain ...
3,3,11759079,ACCOUNTANT,2,SENIOR ACCOUNTANT\nExperience\nCompany Name Ju...,senior accountant experience company name june...,skill developed writing skills drafting forty...
4,4,12065211,ACCOUNTANT,2,SENIOR ACCOUNTANT\nProfessional Summary\nSenio...,senior accountant professional summary senior ...,skill senior accountant professional summary ...


### Classification

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

requiredText = resumeDataSet['Processed_Data'].values
requiredTarget = resumeDataSet['Category'].values

word_vectorizer = TfidfVectorizer(
    max_features=5000,  
        min_df=2,       
        max_df=0.95,      
        ngram_range=(1, 2),
        stop_words='english',
        sublinear_tf=True)

word_vectorizer.fit(requiredText)
WordFeatures = word_vectorizer.transform(requiredText)


X_train,X_test,y_train,y_test = train_test_split(WordFeatures,requiredTarget,random_state=42, test_size=0.2,
                                                 shuffle=True, stratify=requiredTarget)
print(X_train.shape)
print(X_test.shape)

(1984, 5000)
(497, 5000)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

rf_model = OneVsRestClassifier(RandomForestClassifier())
rf_model.fit(X_train, y_train)


y_pred_rf = rf_model.predict(X_test)


print("\nRandomForestClassifier Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))


RandomForestClassifier Results:
Accuracy: 0.7163
Confusion Matrix:
[[24  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  4  3  0  0  0  0]
 [ 0  0  8  0  1  0  0  2  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0]
 [ 1  1  0  8  0  0  0  0  0  1  0  0  0  1  0  1  0  1  0  2  1  1  1  0]
 [ 0  1  1  0  7  0  0  0  0  0  1  0  0  1  0  0  0  0  0  1  1  1  1  6]
 [ 0  0  0  0  0  1  1  1  0  0  0  0  1  0  0  0  1  0  0  1  0  0  1  0]
 [ 0  0  0  0  0  0 21  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 1  0  0  0  0  0  0 11  0  2  0  0  1  0  0  0  4  0  1  1  1  0  1  0]
 [ 0  0  0  0  0  0  0  1  0  1  1  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  0 21  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  1  0  0  0 21  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  1  0  0  0 17  0  0  0  1  0  0  1  0  0  1  0  0]
 [ 2  0  0  0  0  0  1  0  0  1 

In [20]:
test=pre_process_text(resume_text)
text_features = word_vectorizer.transform([test])
pred = rf_model.predict(text_features)
pred[0]


'INFORMATION-TECHNOLOGY'

#### Using Extracted Data

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

requiredText = resumeDataSet['Processed_Feature'].values
requiredTarget = resumeDataSet['Category'].values

word_vectorizer = TfidfVectorizer(
    max_features=5000,  
        min_df=2,       
        max_df=0.95,      
        ngram_range=(1, 2),
        stop_words='english',
        sublinear_tf=True)

word_vectorizer.fit(requiredText)
WordFeatures = word_vectorizer.transform(requiredText)


X_train,X_test,y_train,y_test = train_test_split(WordFeatures,requiredTarget,random_state=42, test_size=0.2,
                                                 shuffle=True, stratify=requiredTarget)
print(X_train.shape)
print(X_test.shape)

(1984, 5000)
(497, 5000)


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

rf_model = OneVsRestClassifier(RandomForestClassifier())
rf_model.fit(X_train, y_train)


y_pred_rf = rf_model.predict(X_test)


print("\nRandomForestClassifier Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))


RandomForestClassifier Results:
Accuracy: 0.7465
Confusion Matrix:
[[24  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 15  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  1  2  2  0  0  0  2]
 [ 0  0  6  0  0  0  0  1  0  0  0  2  0  0  0  0  1  1  0  0  1  1  0  0]
 [ 1  0  0  7  1  0  1  0  0  2  0  0  0  2  1  0  0  0  0  1  2  1  0  0]
 [ 0  0  1  0  5  0  0  0  0  2  0  0  0  1  0  0  0  0  0  1  1  0  1  9]
 [ 1  0  0  0  0  1  2  0  0  0  0  0  1  0  0  1  0  0  1  0  0  0  0  0]
 [ 0  0  0  1  0  0 19  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1  1]
 [ 1  0  0  1  0  0  0 13  0  2  0  0  2  0  1  0  1  0  1  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0 23  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 21  0  0  1  1  0  0  0  0  0  0  0  0  1]
 [ 1  0  0  0  0  0  0  1  0  0  0 18  0  0  1  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0 

In [24]:
test=pre_process_text(resume_text)
text_features = word_vectorizer.transform([test])
pred = rf_model.predict(text_features)
pred[0]


'DIGITAL-MEDIA'

Extracted features lose the context of the resume

### Recommendation & Ranking

In [40]:
df=pd.read_csv("job_title_des.csv")
df["Job Description"] = df["Job Description"].drop_duplicates().dropna()

In [44]:
df["Job Description"]=df["Job Description"].astype(str)
df["Job Description"]

0       We are looking for hire experts flutter develo...
1       PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2       Data Scientist (Contractor)\n\nBangalore, IN\n...
3       JOB DESCRIPTION:\n\nStrong framework outside o...
4       job responsibility full stack engineer – react...
                              ...                        
2272    Job Summary\nPublished on : 26 days ago\nVacan...
2273    business entity cisco umbrella focus cloud-bas...
2274    Urgently reqd in a college in Mohali\nNetwork ...
2275    Key Responsibilities: Team leads for small or ...
2276    leslie hindman auctioneer one nation 's leadin...
Name: Job Description, Length: 2277, dtype: object

In [45]:
df["Job Description"] = df["Job Description"].apply(lambda x: pre_process_text(x))


In [46]:
resumes = resumeDataSet['Processed_Data'].dropna().tolist()
jobs = df['Job Description'].dropna().tolist()

In [47]:
combined = resumes + jobs

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(combined)

resume_vectors = tfidf_matrix[:len(resumes)]
job_vectors = tfidf_matrix[len(resumes):]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
test=vectorizer.transform([resume_text])
similarity_matrix = cosine_similarity(test, job_vectors)


In [57]:

top3_ind = np.argsort(similarity_matrix[0])[-3:][::-1]

for job_rank, job_idx in enumerate(top3_ind, 1):
    job_title = df[df['Job Description'] == jobs[job_idx]]['Job Title'].values[0]
    print(f"Rank {job_rank}: {job_title} (Score: {similarity_matrix[0][job_idx]:.2f})")

Rank 1: Java Developer (Score: 0.12)
Rank 2: Django Developer (Score: 0.12)
Rank 3: Machine Learning (Score: 0.11)
